In [5]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

# Path to the saved model directory
output_dir = r"C:\CADT\Data science\Year 4\NLPner_model"

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(output_dir)
model = AutoModelForTokenClassification.from_pretrained(output_dir)

print("Model and tokenizer loaded successfully!")


c:\CADT\Data science\Year 4\NLP\NLP_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Model and tokenizer loaded successfully!


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification

# Load the saved model and tokenizer
def load_model(output_dir):
    """
    Loads the model and tokenizer from the specified directory.
    """
    tokenizer = AutoTokenizer.from_pretrained(output_dir)
    model = AutoModelForTokenClassification.from_pretrained(output_dir)
    return model, tokenizer

# Perform NER predictions
def predict_ner(text, model, tokenizer, id2label):
    """
    Performs NER on the given text using the trained model.
    Args:
        text (str): The input text for NER.
        model: The trained NER model.
        tokenizer: Tokenizer corresponding to the model.
        id2label (dict): Mapping of label IDs to label names.

    Returns:
        List of tuples: Each tuple contains a token and its predicted label.
    """
    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    
    # Get model predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Process outputs
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=2).squeeze().tolist()
    
    # Map tokens to labels
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"].squeeze().tolist())
    labels = [id2label[pred] for pred in predictions]

    # Exclude special tokens ([CLS], [SEP])
    predictions_with_labels = [
        (token, label) for token, label in zip(tokens, labels) 
        if token not in tokenizer.all_special_tokens
    ]
    return predictions_with_labels

# Command-line interface for NER
def ner_prediction_interface(model, tokenizer, id2label):
    """
    Provides a CLI interface for inputting sentences and predicting NER labels.
    Args:
        model: The trained NER model.
        tokenizer: Tokenizer corresponding to the model.
        id2label (dict): Mapping of label IDs to label names.
    """
    print("Named Entity Recognition (NER) Prediction Interface")
    print("Type 'exit' to quit.")
    
    while True:
        # Input sentence
        text = input("\nEnter a sentence for NER prediction: ")
        if text.lower() == "exit":
            print("Exiting... Goodbye!")
            break
        
        # Perform predictions
        predictions = predict_ner(text, model, tokenizer, id2label)
        
        # Display results
        print("\nPredicted NER Labels:")
        for token, label in predictions:
            print(f"{token}: {label}")

# Main function
if __name__ == "__main__":
    # Path to your saved model
    # output_dir = "./ner_model"
    output_dir = r"C:\CADT\Data science\Year 4\NLPner_model"

    # Define label mapping (Replace with actual labels)
    id2label = {0: "O", 1: "B", 2: "I"}  # Update with your label mapping
    
    # Load the model and tokenizer
    model, tokenizer = load_model(output_dir)
    print("Model and tokenizer loaded successfully!")

    # Launch the prediction interface
    ner_prediction_interface(model, tokenizer, id2label)


Model and tokenizer loaded successfully!
Named Entity Recognition (NER) Prediction Interface
Type 'exit' to quit.

Predicted NER Labels:
produces: O
natural: B
written: I
or: O
spoken: I
language: I
from: O
structured: B
and: O
un: B
##st: B
##ructured: B
data: I

Predicted NER Labels:
produces: O
natural: B
written: I
or: O
spoken: I
language: I
from: O
structured: B
and: O
un: B
##st: B
##ructured: B
data: I

Predicted NER Labels:
this: O
blog: O
discusses: O
the: O
different: O
nl: B
##p: B
techniques: O
and: O
tasks: O
.: O
we: O
will: O
be: O
using: O
python: B
code: I
to: O
demo: O
what: O
and: O
how: O
each: O
task: O
works: O
.: O

Predicted NER Labels:

Predicted NER Labels:

Predicted NER Labels:

Predicted NER Labels:

Predicted NER Labels:

Predicted NER Labels:

Predicted NER Labels:

Predicted NER Labels:

Predicted NER Labels:

Predicted NER Labels:


In [1]:
%pip install streamlit


  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
   ---------------------------------------- 0.0/9.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.1 MB ? eta -:--:--
   --- ------------------------------------ 0.8/9.1 MB 1.5 MB/s eta 0:00:06
   ---- ----------------------------------- 1.0/9.1 MB 1.6 MB/s eta 0:00:06
   ------ --------------------------------- 1.6/9.1 MB 1.8 MB/s eta 0:00:05
   --------- ------------------------------ 2.1/9.1 MB 1.9 MB/s eta 0:00:04
   ----------- ---------------------------- 2.6/9.1 MB 2.0 MB/s eta 0:00:04
   ------------- -------------------------- 3.1/9.1 MB 2.1 MB/s eta 0:00:03
   ----------------- ---------------------- 3.9/9.1 MB 2.2 MB/s eta 0:00:03
   ------------------- -------------------- 4.5/9.1 MB 2.3 MB/s eta 0:00:02
   -------------------- ------------------- 4.7/9.1 MB 2.2 MB/s eta 0:00:02
   ----------------------- ---------------- 5

In [3]:
import streamlit as st
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification

# Path to the saved model and tokenizer
output_dir = r"NLPner_model"

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(output_dir)
model = AutoModelForTokenClassification.from_pretrained(output_dir)

# Define your id2label mapping (adjust based on your model's labels)
id2label = {0: "O", 1: "B", 2: "I"}  # Adjust to match your model's label IDs

# Function to predict NER labels
def predict_ner(text, model, tokenizer, id2label):
    """
    Predicts NER labels for the input text using the model and tokenizer.
    """
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    
    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)

    # Get predicted labels
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=2).squeeze().tolist()

    # Convert token IDs to tokens
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"].squeeze().tolist())

    # Map predictions to labels
    labels = [id2label[pred] for pred in predictions]

    # Filter out special tokens
    predictions_with_labels = [
        (token, label) for token, label in zip(tokens, labels) if token not in tokenizer.all_special_tokens
    ]
    
    return predictions_with_labels

# Streamlit Interface
def run_streamlit_interface():
    # Set page configuration
    st.set_page_config(page_title="NER Prediction", layout="wide")
    
    # Add a custom header and subtitle
    st.markdown("<h1 style='text-align: center; color: #4CAF50;'>Named Entity Recognition (NER)</h1>", unsafe_allow_html=True)
    st.markdown("<h3 style='text-align: center;'>Predict entity labels from a given sentence</h3>", unsafe_allow_html=True)

    # Styled input box for the sentence
    text = st.text_area(
        "Enter a Sentence for NER Prediction:",
        placeholder="Type your sentence here...",
        height=150,
        max_chars=500,
        help="Enter a sentence, and the model will predict the named entities in it."
    )

    # Apply some styling to buttons
    button_style = """
    <style>
        .stButton>button {
            background-color: #4CAF50;
            color: white;
            border-radius: 10px;
            padding: 10px 30px;
            font-size: 16px;
            font-weight: bold;
        }
    </style>
    """
    st.markdown(button_style, unsafe_allow_html=True)

    # Create a custom button to trigger the prediction
    if st.button("Predict NER"):
        if text.strip():  # Check if the text is not empty
            with st.spinner('Predicting...'):
                # Get predictions
                predictions = predict_ner(text, model, tokenizer, id2label)

                # Display the predictions in a nicely formatted list
                st.subheader("Prediction Results:")
                st.markdown("<h4 style='text-align: center;'>Entities Detected:</h4>", unsafe_allow_html=True)
                
                # Using a simple list format for displaying predictions
                for token, label in predictions:
                    st.markdown(f"<b>{token}</b>: {label}", unsafe_allow_html=True)
        else:
            st.warning("Please enter a sentence to predict.")

# Run the Streamlit app
if __name__ == "__main__":
    run_streamlit_interface()


2024-12-16 14:58:56.788 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-16 14:58:56.789 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-16 14:58:56.920 
  command:

    streamlit run c:\CADT\Data science\Year 4\NLP\NLP_env\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-12-16 14:58:56.921 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-16 14:58:56.922 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-16 14:58:56.923 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-16 14:58:56.924 Thread 'MainThread': missing ScriptRunContext! This warning can be ig